In [1]:
import sys
sys.path.insert(0, "..")

import numpy as np
from matplotlib import pyplot as plt

from module.base.network import Network
from module.simulation.meanfield import MeanField
import module.components.CONST as CONST
from module.components.lawrence_dist import LawrenceDist

In [2]:
ld = LawrenceDist()

In [3]:
np.sum(ld.phase_space * ld.calc_prob(0.12345678987654))

0.12345678987654